## Abstract
Our project predicts the match results by using historical ODI cricket data and state-of-the-art machine learning. It redefines cricket analysis by closely monitoring forecast accuracy and encouraging user engagement. The primary goal of our project is to use historical and current match data, train them under regression classification models, select the best model and then use it to predict the winner of a cricket match based on the live match statistics (e.g., runs scored, wickets fallen, overs bowled), that update at regular intervals. This model will be displayed on a webpage which will refresh every time a new stat is updated, and the prediction from the model will update.


## Introduction
For this project we wanted to showcase our prediction skill-set with the challenge of a live dataset, and what better live dataset to use than sports, so we went with our favorite sport - cricket. The past data will include information of ODI Matches from years 2002 to 2022, while the live data will consist of data from the year 2023. Each entry from 2023 will be read from the actual CSV file and entered into a database table with an interval of 10 to 20 seconds between two consecutive entries. These entries will be considered as live data and will be sent to the API caller.

We’ll use two datasets:

ODI_Match_Data.csv: Provides facts about the location and season of the cricket matches along with team information and the play results from each team member. We’ll need this one to investigate partnerships between batsmen. It’s dimensions are 155432 rows of data by 23 variable columns. The data that appears in this proposal is a truncated version for ease of storage, but the project will utilize an API that will supply the entire dataset.

ODI_Match_info.csv: Overlaps in data with the above but provides information on the umpire, performance, and the city the match took place. We’ll need this one to analyze the batting and bowling performance of each player. It’s dimensions are 2380 rows of data by 18 variable columns.

## Setup

In [ ]:
import pandas as pd
import glob
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split as tts
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier  
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import pickle

## EDA
Here, we first carried out Data Preprocessing where we merged the match_data and match_info files, then carried out Exploratory Data Analysis by using pairplots to find correlations between different variables, if any.


In [ ]:
info = pd.read_csv('data/ODI_Match_info.csv')
info = info.rename(columns = {'id':'match_id'})

#append all files together
csv_files = ['data/output_1.csv','data/output_2.csv','data/output_3.csv','data/output_4.csv','data/output_5.csv','data/output_6.csv','data/output_7.csv','data/output_8.csv','data/output_9.csv']

matchData = pd.concat([pd.read_csv(f,low_memory=False) for f in csv_files ], ignore_index=True)

#merge frames on match ID column

totalData = pd.merge(matchData, info, on = 'match_id') #merge by identical column 'match_id'
totalData.drop(totalData.filter(regex='_y$').columns, axis=1, inplace=True) #drop duplicate columns

totalData = totalData.rename(columns = {'season_x':'season', 'venue_x':'venue'})

from02to22 = totalData[~totalData['season'].astype(str).str.startswith(('2023/2024','2023', '2022/23'))] #exclude 2023 data

from02to22
print(type(from02to22)) #confirm data is read in as a df
print(from02to22.shape) #confirm data shape
print(from02to22.dtypes) #understand the types of data in the df
print(from02to22.isna().sum()) #count NA values in columns
print(pd.DataFrame.describe(from02to22)) #descriptive function to look at dataframe)


winners = sns.countplot(data = from02to22, y = 'winner', order=from02to22['winner'].value_counts().index)
winners

# corr = sns.pairplot(from02to22)
# corr


## Data Cleaning
Here, during a thorough process of Data Cleaning, we dropped some columns which were largely empty and replaced some empties with 'Unknown'.

In [ ]:
#drop columns that have more than 1Million NaNs

colNaCounts = from02to22.isna().sum()


columns_to_drop = colNaCounts[colNaCounts >= 1000000].index.tolist()

# Drop identified columns from the DataFrame
from02to22 = from02to22.drop(columns=columns_to_drop)


#revalue new winner column

from02to22['winnerTeam'] = from02to22.apply(lambda row: 'team1' if row['winner'] == row['team1'] else 'team2', axis=1)






#convert Nan cities to 'Unknown'
#drop winner NA columns
#convert NA player of match to 'unknown'
#convert NA umpire 3 to 'unknown'

from02to22['city'] = from02to22['city'].fillna('Unknown') 
from02to22['player_of_match'] = from02to22['player_of_match'].fillna('Unknown') 
from02to22['umpire3'] = from02to22['umpire3'].fillna('Unknown') 
from02to22 = from02to22.dropna(subset=['winner'])
from02to22 = from02to22.drop(columns = ['match_id', 'start_date', 'date', 'winner', 'cricsheet_id', 'season', 'venue', 'city', 'player_of_match', 'win_by_runs', 'win_by_wickets', 'umpire1', 'umpire2', 'umpire3', 'result']) #is date specific data really useful? also drop continuous match identifiers. We want the match stats
# corr = sns.pairplot(from02to22)
# corr

## Feature Engineering
The next step is the Feature Engineering where we:
 - Normalized the continuous data
 - Encoded categorical variables

In [ ]:
#normalize continuous data because we dont have a normal distribution

to_norm = (from02to22.select_dtypes(include =['int64', 'float64'])) #select continuous variables

continuous = MinMaxScaler().fit_transform(to_norm) #fit and transform min max scaler (normalizes)
continuous = pd.DataFrame(continuous, index = from02to22.index, columns = list(to_norm))

#encode categorical variables

to_encode = (from02to22.select_dtypes(include =['object'])) #test if anything is an object or category variable

label_encoders = [] #make new encoder for each column
encoded_data = pd.DataFrame()

encoding_dicts = {}

# Iterate over each column in 'to_encode' and encode using a separate LabelEncoder
for col in to_encode:
    # Create a new instance of LabelEncoder for the current column
    encoder = LabelEncoder()
    
    # Fit and transform the data in 'from02to22[col]'
    encoded_data[col] = encoder.fit_transform(from02to22[col])
    
    # Store the encoder in the list
    label_encoders.append(encoder)

    encoding_dicts[col] = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))


#patch the columns back together

data = pd.concat([encoded_data.reset_index(drop=True), continuous.reset_index(drop=True)], axis = 1) #reset indices to avoid errors in concat


## Feature Selection
As we have eliminated the unnecessary columns from data previously through data preprocessing and cleaning, now we directly select the top 10 columns except winnerTeam as the input features of the model and the winnerTeam itself as the target feature. 

In [ ]:
X = data.drop('winnerTeam', axis = 1) #input
y = data['winnerTeam'] #target

selector = SelectKBest(score_func= f_classif, k = 10)
top10 = selector.fit_transform(X,y) #create variable that is the top 10 best feautures
cols_idxs = selector.get_support(indices=True) #grab indices from feature cols, get_support is from **sklearn**
top10 = X.iloc[:,cols_idxs] #add columns from whole dataset to the selected columns dataset https://stackoverflow.com/questions/39839112/the-easiest-way-for-getting-feature-names-after-running-selectkbest-in-scikit-le
print(top10)

## Model Training (Logistic Regression)
The input features and target feature are now broken down into training and testing groups and are put under our first regression model - Logistic regression.

In [ ]:
#split data into training and testing
X_train, X_test, y_train, y_test = tts(top10, y, test_size = 0.2) #break into 4 groups for testing and training, make the training dataset 70% of the data and the testing dataset 30% https://builtin.com/data-science/train-test-split
#PCA
# pca = PCA(n_components = 5) #go from 10 components to 5 components
# X_train_pca = pca.fit_transform(X_train)
# X_test_pca = pca.transform(X_test)

# X_train = X_train_pca
# X_test = X_test_pca

#training and testing models

 #decision tree
# lr = LogisticRegression()

# lr.fit(X_train, 
#           y_train)
# predictLR = lr.predict(X_test)
# outcomeLR = pd.DataFrame ({'Actual': y_test, 'Predicted': predictLR})
# outcomeLR['Actual'] = label_encoders[9].inverse_transform(outcomeLR['Actual'])
# outcomeLR['Predicted'] = label_encoders[9].inverse_transform(outcomeLR['Predicted'])

# key = (label_encoders[9].classes_)


## Model Validation
The results of Logistic regression are now cross validated and their performance metrics are displayed. As we observe the accuracy and precision of this model is around 55% and the area under ROC curve is also around 0.5. As these metrics show that this might not be the best model, we move forward and train another model.

In [ ]:
#Evaluating models
#logistic regression
cvLR = cross_validate(lr,X_train, y_train)
print("cross validation of logistic regression", cvLR)
print("accuracy of logistic regression:", metrics.accuracy_score(y_test, predictLR)) #testing how accuracy of the models
print("precision of logistic regression:", metrics.precision_score(y_test, predictLR, average = 'weighted'))
print("recall of logistic regression:", metrics.recall_score(y_test, predictLR, average = 'weighted'))
print("f1 of logistic regression:", metrics.f1_score(y_test, predictLR, average = 'weighted'))
print("ROCAUC macro of logistic regression:", metrics.roc_auc_score(y_test, predictLR))
print("ROCAUC micro of logistic regression:", metrics.roc_auc_score(y_test, predictLR, average = 'micro'))


y_pred_proba = lr.predict_proba(X_test)[::,1] #https://www.statology.org/plot-roc-curve-python/
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
plt.plot(fpr,tpr)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.title('ROC Curve Logistic Regression')
plt.show()

print('')


### Random Forest Classifier
Now we take the Random Forest  Classifier, cross validate its results and display its performance metrics. As we observe the accuracy and precision of this model is around 96% and the area under ROC curve is also around 0.96. These metrics clearly show that this is the best model, so we move forward with this model.

In [ ]:
#random forest
rfc = RandomForestClassifier(criterion='gini')
rfc.fit(X_train, 
          y_train)
predictRFC = rfc.predict(X_test)
outcomeRFC = pd.DataFrame ({'Actual': y_test, 'Predicted': predictRFC})
outcomeRFC['Actual'] = label_encoders[9].inverse_transform(outcomeRFC['Actual'])
outcomeRFC['Predicted'] = label_encoders[9].inverse_transform(outcomeRFC['Predicted'])

#Evaluating models
#random forest
cvRF = cross_validate(rfc,X_train, y_train)
print("cross validation of random forest", cvRF)
print("accuracy of random forest:", metrics.accuracy_score(y_test, predictRFC)) #testing how accuracy of the models
print("precision of random forest:", metrics.precision_score(y_test, predictRFC, average = 'weighted'))
print("recall of random forest:", metrics.recall_score(y_test, predictRFC, average = 'weighted'))
print("f1 of random forest:", metrics.f1_score(y_test, predictRFC, average = 'weighted'))
print("ROCAUC macro of random forest:", metrics.roc_auc_score(y_test, predictRFC))
print("ROCAUC micro of random forest:", metrics.roc_auc_score(y_test, predictRFC, average = 'micro'))


y_pred_proba = rfc.predict_proba(X_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
plt.plot(fpr,tpr)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.title('ROC Curve Random Forest')
plt.show()

#FOR LOCAL USE
# Step 2: Save the trained model to a pickle file
# with open("/Users/cortmann/Desktop/523 - Data Mining/localCricketPredictions/cricketPrediction.pkl", "wb") as f:
#     pickle.dump(rfc, f)

## Feed 2023 Data
Now we feed the live data (which is the 2023 match data), through back end API, at regular intervals, thereby changing the stats everytime. And each time a statistic is updated, the prediction updates as well.

In [ ]:
# Step 2: Save the trained model to a pickle file
# with open("data/cricketPrediction.pkl", "wb") as f:
#     pickle.dump(rfc, f)
    


# Step 3: Load the model from the pickle file

# with open("linear_regression_model.pkl", "rb") as f:
#     loaded_model = pickle.load(f)


## Model Implementation
Finally, we display the model predictionm on a webpage and take you through our journey of prediction!


In [ ]:
import requests

url = "https://goto-brought-auction-deck.trycloudflare.com/analyzingtrends/getmatchbyid/1"  
response = requests.get(url)

if response.status_code == 200:  # Check if the request was successful
    json_data = response.json()  # Convert response to JSON format
    print(json_data['season'])  # Print the JSON data
else:
    print("Failed to retrieve data from the API.")

In [ ]:
loaded_model = None

# Load the model from the pickle file
with open("/Users/cortmann/Desktop/523 - Data Mining/localCricketPredictions/cricketPrediction.pkl", "rb") as f:
    loaded_model = pickle.load(f)
    
predictRFC = loaded_model.predict(X_test)

outcomeRFC = pd.DataFrame ({'Actual': y_test, 'Predicted': predictRFC})
outcomeRFC['Actual'] = label_encoders[9].inverse_transform(outcomeRFC['Actual'])
outcomeRFC['Predicted'] = label_encoders[9].inverse_transform(outcomeRFC['Predicted'])
    


# cvRF = cross_validate(loaded_model,X_train, y_train)
# print("cross validation of random forest", cvRF)
# print("accuracy of random forest:", metrics.accuracy_score(y_test, predictRFC)) #testing how accuracy of the models
# print("precision of random forest:", metrics.precision_score(y_test, predictRFC, average = 'weighted'))
# print("recall of random forest:", metrics.recall_score(y_test, predictRFC, average = 'weighted'))
# print("f1 of random forest:", metrics.f1_score(y_test, predictRFC, average = 'weighted'))
# print("ROCAUC macro of random forest:", metrics.roc_auc_score(y_test, predictRFC))
# print("ROCAUC micro of random forest:", metrics.roc_auc_score(y_test, predictRFC, average = 'micro'))


# y_pred_proba = loaded_model.predict_proba(X_test)[::,1]
# fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
# plt.plot(fpr,tpr)
# plt.ylabel('True Positive Rate')
# plt.xlabel('False Positive Rate')
# plt.title('ROC Curve Random Forest')
# plt.show()
